In [2]:
import pandas as pd
import re
from abctool import abctool, abctune

In [3]:
df = pd.read_csv('/home/devcontainers/git/Tune_Similarity/raw_data/tunes.csv')
df.head()

,tune_id,setting_id,name,type,meter,mode,abc,date,username
0,18105,35234,"$150 Boot, The",polka,2/4,Gmajor,|:d>g fe|dB AG|E/F/G E/F/G|BA GF|\nd>g fe|dB A...,2019-07-06 04:39:09,NfldWhistler
1,11931,11931,'G Iomain Nan Gamhna,slip jig,9/8,Gmajor,dBB B2 A BAG|dBB Bcd efg|dBB B2 A BAG|eAA dBG ...,2012-05-17 07:49:26,iliketurtles
2,11931,48772,'G Iomain Nan Gamhna,slip jig,9/8,Amixolydian,|:dBB BBA BAG|dBB Bcd efg|dBB BBA BAG|e2A dBG ...,2023-11-25 22:54:00,birlibirdie
3,15326,28560,'S Ann An Ìle,strathspey,4/4,Gmajor,|:G>A B>G c>A B>G|E<E A>G F<D D2|G>A B>G c>A B...,2016-03-31 15:34:45,danninagh
4,15326,28582,'S Ann An Ìle,strathspey,4/4,Gmajor,"uD2|:{F}v[G,2G2]uB>ud c>A B>G|{D}E2 uA>uG F<D ...",2016-04-03 09:15:08,DonaldK


In [4]:
df["full_abc"] = df.apply(
    lambda row: f"""X:{row.setting_id}
T:{row["name"]}
M:{row.meter}
K:{row["mode"]}
{row.abc}
""",
    axis=1
)

In [5]:
def get_transpose_distance(m):
    # m - mode of the tune
    # returns distance to transpose so that tonic is C
    # proper way to do this would be to do regex to match any # or b, but we don't have those keys in the dataset
    tonic = m[0]
    # again, being lazy
    distance = {
        "A" : 3,
        "B" : 1,
        "C" : 0,
        "D" : -2,
        "E" : -4,
        "F" : -5,
        "G" : -7
    }
    return distance[tonic]



def transpose_row(row):
    m = row["mode"]
    abc = row["full_abc"]
    tune = abctune(abc)
    d = get_transpose_distance(m)
    tune.transpose(d)
    return tune.getAbc()




In [7]:
# df["transposed_abc"] = df.apply(transpose_row, axis = 1)

def safe_transpose_row(row):
    try:
        return transpose_row(row)
    except Exception as e:
        print(f"Error on row {row.name}: {e}")
        return None

df["transposed_abc"] = df.apply(safe_transpose_row, axis=1)

Error on row 2443: '^=c' is not in list
Error on row 5372: 'NoneType' object has no attribute 'span'
Error on row 11878: Invalid key: Embj
Error on row 19267: '==c' is not in list
Error on row 21336: '^=c' is not in list
Error on row 27181: 'NoneType' object has no attribute 'span'
Error on row 28447: 'F#b' is not in list
Error on row 30567: 'int' object is not subscriptable
Error on row 30571: can only concatenate str (not "int") to str
Error on row 37764: '^^e' is not in list


: 